In [1]:
# working with the new york times headlines 
# subjectivity vs. sentiment might not be in here (objective headlines)
# Keyword trends over time 

In [2]:
import json
import os
import pandas as pd

In [27]:
# read in, flatten and clean any data 

path = '/Users/philazar/Desktop/projects/Mar21-vaccine-uptake/data'
vax = 'nyt-term-vaccine2021.json'
covid = 'nyt-term-covid2021.json'
with open(f'{path}/{vax}') as f:
    vax_df = json.load(f)
          
with open(f'{path}/{covid}') as f:
    covid_df = json.load(f)

In [28]:
covid_response = covid_df.get('response')
vax_response = vax_df.get('response')

In [29]:

covid_df = pd.json_normalize(covid_response.get('docs'))
covid_df = covid_df[['pub_date','abstract', 'web_url', 'snippet', 'lead_paragraph', 'print_section', 'print_page', 'source', 'headline.main', 'headline.print_headline']]


vax_df = pd.json_normalize(vax_response.get('docs'))
vax_df = vax_df[['pub_date','abstract', 'web_url', 'snippet', 'lead_paragraph', 'print_section', 'print_page', 'source', 'headline.main', 'headline.print_headline']]


In [31]:
pd.options.display.width = 0
vax_df.lead_paragraph

0    New Yorkers living in public housing have stru...
1    First, the good news for America: People are g...
2    It is only natural that the American governmen...
3    Prime Minister Justin Trudeau announced on Fri...
4    I’m not usually given to bursting into tears i...
5    The next major flash point over coronavirus re...
6    Canada’s drug regulator authorized the AstraZe...
7    BERLIN — At the start of the year, many German...
8    Do the vaccines against the coronavirus offer ...
9    This Op-Ed has been updated to reflect news de...
Name: lead_paragraph, dtype: object

In [35]:
covid_df['keyword'] = 'covid'
vax_df['keyword'] = 'vaccine'
total_df = pd.concat([covid_df, vax_df], ignore_index=True)

In [36]:
total_df.head()

,pub_date,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,headline.main,headline.print_headline,keyword
0,2021-02-08T14:43:09+0000,New Yorkers living in public housing have stru...,https://www.nytimes.com/interactive/2021/02/08...,New Yorkers living in public housing have stru...,New Yorkers living in public housing have stru...,A,6,The New York Times,These New Yorkers Are Signing Up Their Neighbo...,These New Yorkers Are Signing Up Their Neighbo...,covid
1,2021-03-02T23:20:03+0000,"A century ago, another pandemic terrorized the...",https://www.nytimes.com/2021/03/02/opinion/cov...,"A century ago, another pandemic terrorized the...",This article is part of the Debatable newslett...,NaN,NaN,The New York Times,The Case for Covid Optimism,None,covid
2,2021-02-26T17:44:09+0000,But does it work?,https://www.nytimes.com/2021/02/26/well/live/a...,But does it work?,"March 2, 2021: This article has been updated t...",D,6,The New York Times,At-Home Covid Testing Is Here,Do Your Testing For Covid at Home,covid
3,2021-02-25T20:41:08+0000,Hundreds of thousands of AstraZeneca doses are...,https://www.nytimes.com/2021/02/25/world/europ...,Hundreds of thousands of AstraZeneca doses are...,"BERLIN — At the start of the year, many German...",A,4,The New York Times,"Germans Clamor for Covid Vaccines, but Shun As...",They Wanted a Covid Vaccine. But Not That One.,covid
4,2021-02-19T11:00:08+0000,As the United States continues its campaign to...,https://www.nytimes.com/2021/02/19/podcasts/th...,As the United States continues its campaign to...,"For American adults, the conclusion of the pan...",NaN,NaN,The New York Times,Children and Covid,None,covid


In [37]:
# small data, types of analysis: keyword count, bag of words over time? 

import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/philazar/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [38]:
stop_words = set(stopwords.words("english"))

In [43]:
# need to make the words into a list 
# using lead paragraph (more words)
from datetime import datetime
total_df['clean_date'] = total_df['pub_date'].apply(lambda x: datetime.strftime(datetime.strptime(x[0:10], '%Y-%m-%d'),'%Y-%m-%d' ))

In [44]:
total_df.head()

,pub_date,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,headline.main,headline.print_headline,keyword,clean_date
0,2021-02-08T14:43:09+0000,New Yorkers living in public housing have stru...,https://www.nytimes.com/interactive/2021/02/08...,New Yorkers living in public housing have stru...,New Yorkers living in public housing have stru...,A,6,The New York Times,These New Yorkers Are Signing Up Their Neighbo...,These New Yorkers Are Signing Up Their Neighbo...,covid,2021-02-08
1,2021-03-02T23:20:03+0000,"A century ago, another pandemic terrorized the...",https://www.nytimes.com/2021/03/02/opinion/cov...,"A century ago, another pandemic terrorized the...",This article is part of the Debatable newslett...,NaN,NaN,The New York Times,The Case for Covid Optimism,None,covid,2021-03-02
2,2021-02-26T17:44:09+0000,But does it work?,https://www.nytimes.com/2021/02/26/well/live/a...,But does it work?,"March 2, 2021: This article has been updated t...",D,6,The New York Times,At-Home Covid Testing Is Here,Do Your Testing For Covid at Home,covid,2021-02-26
3,2021-02-25T20:41:08+0000,Hundreds of thousands of AstraZeneca doses are...,https://www.nytimes.com/2021/02/25/world/europ...,Hundreds of thousands of AstraZeneca doses are...,"BERLIN — At the start of the year, many German...",A,4,The New York Times,"Germans Clamor for Covid Vaccines, but Shun As...",They Wanted a Covid Vaccine. But Not That One.,covid,2021-02-25
4,2021-02-19T11:00:08+0000,As the United States continues its campaign to...,https://www.nytimes.com/2021/02/19/podcasts/th...,As the United States continues its campaign to...,"For American adults, the conclusion of the pan...",NaN,NaN,The New York Times,Children and Covid,None,covid,2021-02-19


In [45]:
# save data
total_df.to_csv(f'{path}/nyt_headlines_flat.csv')

In [52]:
lead_paragraphs = total_df['lead_paragraph'].to_list()

In [59]:
# take out stop words and lemmetize 
nltk.download('punkt')
tokens = [nltk.word_tokenize(text) for text in lead_paragraphs]

[nltk_data] Downloading package punkt to /Users/philazar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [72]:
nltk.download('wordnet')
import string
lem = nltk.stem.WordNetLemmatizer()
clean_tokens = []
for pg in tokens:
    new = [lem.lemmatize(word) for word in pg if word not in stop_words]
    new_ =[n for n in new if n not in string.punctuation]
    clean_tokens.append(new_)
    

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/philazar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [92]:
# perform sentiment 
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

compounds = []
negative = []
positive = []
neutral = []
for pg in clean_tokens:
    sentence = ' '.join(pg)
    scores = sia.polarity_scores(sentence)
    compounds.append(scores['compound'])
    negative.append(scores['neg'])
    positive.append(scores['pos'])
    neutral.append(scores['neu'])
    
    
    

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/philazar/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [90]:
scores

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [94]:
total_df['compound_sentiment'] = compounds
total_df['neg_sentiment'] = negative
total_df['pos_sentiment'] = positive 
total_df['neutral_sentiment'] = neutral

In [96]:
# sentiment is mostly neutral as expected
total_df.head(20)

,pub_date,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,headline.main,headline.print_headline,keyword,clean_date,compound_sentiment,neg_sentiment,pos_sentiment,neutral_sentiment
0,2021-02-08T14:43:09+0000,New Yorkers living in public housing have stru...,https://www.nytimes.com/interactive/2021/02/08...,New Yorkers living in public housing have stru...,New Yorkers living in public housing have stru...,A,6,The New York Times,These New Yorkers Are Signing Up Their Neighbo...,These New Yorkers Are Signing Up Their Neighbo...,covid,2021-02-08,-0.0772,0.130,0.114,0.757
1,2021-03-02T23:20:03+0000,"A century ago, another pandemic terrorized the...",https://www.nytimes.com/2021/03/02/opinion/cov...,"A century ago, another pandemic terrorized the...",This article is part of the Debatable newslett...,NaN,NaN,The New York Times,The Case for Covid Optimism,None,covid,2021-03-02,0.0000,0.000,0.000,1.000
2,2021-02-26T17:44:09+0000,But does it work?,https://www.nytimes.com/2021/02/26/well/live/a...,But does it work?,"March 2, 2021: This article has been updated t...",D,6,The New York Times,At-Home Covid Testing Is Here,Do Your Testing For Covid at Home,covid,2021-02-26,0.0000,0.000,0.000,1.000
3,2021-02-25T20:41:08+0000,Hundreds of thousands of AstraZeneca doses are...,https://www.nytimes.com/2021/02/25/world/europ...,Hundreds of thousands of AstraZeneca doses are...,"BERLIN — At the start of the year, many German...",A,4,The New York Times,"Germans Clamor for Covid Vaccines, but Shun As...",They Wanted a Covid Vaccine. But Not That One.,covid,2021-02-25,-0.4019,0.260,0.144,0.596
4,2021-02-19T11:00:08+0000,As the United States continues its campaign to...,https://www.nytimes.com/2021/02/19/podcasts/th...,As the United States continues its campaign to...,"For American adults, the conclusion of the pan...",NaN,NaN,The New York Times,Children and Covid,None,covid,2021-02-19,0.0000,0.000,0.000,1.000
5,2021-02-23T10:00:12+0000,We might be able to achieve normalcy by summer...,https://www.nytimes.com/2021/02/23/opinion/cov...,We might be able to achieve normalcy by summer...,Christmas of 2021: According to both President...,NaN,NaN,The New York Times,The Covid Emergency Must End,None,covid,2021-02-23,0.3818,0.097,0.194,0.709
6,2021-03-05T01:10:18+0000,"Even within political parties, governors were ...",https://www.nytimes.com/2021/03/04/us/covid-st...,"Even within political parties, governors were ...",Despite President Biden’s sharp criticism of T...,A,1,The New York Times,"As Biden Urges Caution on Covid, Governors Spl...",States Go Own Ways on Covid Rules,covid,2021-03-05,-0.7430,0.289,0.000,0.711
7,2021-02-25T16:45:31+0000,The growing attraction of short-term real esta...,https://www.nytimes.com/2021/02/25/business/re...,The growing attraction of short-term real esta...,"The pandemic forced owners of hotels, malls an...",B,3,The New York Times,Landlords Eager for Covid-Era Revamps Are Find...,Covid-Era Revamps Are Finding Hungry Investors,covid,2021-02-25,-0.8369,0.229,0.000,0.771
8,2021-03-01T20:14:58+0000,The condition is becoming more common as immun...,https://www.nytimes.com/2021/03/01/health/covi...,The condition is becoming more common as immun...,Coronavirus vaccinations can cause enlarged ly...,A,9,The New York Times,"A Covid Vaccine Side Effect, Enlarged Lymph No...",A Covid Vaccine Side Effect Can Be Mistaken fo...,covid,2021-03-01,-0.7845,0.385,0.000,0.615
9,2021-02-26T10:00:03+0000,Test your knowledge of this week’s health news.,https://www.nytimes.com/interactive/2021/02/26...,Test your knowledge of this week’s health news.,Test your knowledge of this week’s health news.,NaN,NaN,The New York Times,"Weekly Health Quiz: Diet, Weight and Covid",None,covid,2021-02-26,0.0000,0.000,0.000,1.000
